# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data 

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import cartopy
import folium
import geoviews


# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,surbo,40.3994,18.1331,23.34,68,1,5.68,IT,1685210577
1,1,haiku-pauwela,20.9219,-156.3051,24.47,74,21,1.34,US,1685210577
2,2,stanley,54.8680,-1.6985,18.77,60,20,5.14,GB,1685210577
3,3,viedma,-40.8135,-62.9967,11.71,28,0,6.92,AR,1685210438
4,4,tolanaro,-25.0319,46.9987,23.04,94,75,5.14,MG,1685210578


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
%%capture --no-display

import hvplot.pandas  # Import hvPlot for Pandas
from bokeh.models import HoverTool

# Configure the map plot_1 with points
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size="Humidity",
    alpha=0.6,
    geo=True,
    tiles="OSM",
    hover_cols=["City", "Humidity"],
    marker="o",
    color="black",
    xlim=(-180, 180),  # Set the x-axis limits to show the whole world
    ylim=(-90, 90),   # Set the y-axis limits to show the whole world
    formatters={"Humidity": "%.2f%"},  # Format the Humidity column with 2 decimal places and "%"
).opts(
    width=800,  # Set the width of the plot to 800 pixels
    height=600  # Set the height of the plot to 600 pixels
)

# Configure the HoverTool to display only the city name and humidity in the pop-up
hover = HoverTool(tooltips=[("City", "@City"), ("Humidity", "@Humidity{0.00}%")], mode='vline')

# Add the HoverTool to the plot
map_plot_1 = map_plot_1.opts(tools=[hover])

# Display the map plot_1
map_plot_1








:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [18]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter the cities based on temperature, wind speed, and cloudiness criteria
filtered_cities = city_data_df[
    (city_data_df["Max Temp"] < 27) &
    (city_data_df["Max Temp"] > 21) &
    (city_data_df["Wind Speed"] < 4.5) &
    (city_data_df["Cloudiness"] == 0)
]

# Configure the filtered map plot
filtered_map_plot = filtered_cities.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM"
)

# Display the filtered map plot
filtered_map_plot



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]

### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create a DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Create a copy of the selected columns from city_data_df
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column named "Hotel Name"
hotel_df = hotel_df.assign(Hotel_Name='')

# Display the updated hotel_df DataFrame
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel_Name
0,surbo,IT,40.3994,18.1331,68,
1,haiku-pauwela,US,20.9219,-156.3051,74,
2,stanley,GB,54.8680,-1.6985,60,
3,viedma,AR,-40.8135,-62.9967,28,
4,tolanaro,MG,-25.0319,46.9987,94,
...,...,...,...,...,...,...
545,diego de almagro,CL,-26.3667,-70.0500,14,
546,el reno,US,35.5323,-97.9550,57,
547,shingu,JP,33.7333,135.9833,84,
548,lufkin,US,31.3382,-94.7291,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000  

params = {
    "categories":"accommodation.hotel",  # Specify the type of place to search (in this case, lodging/hotel)
    "apiKey": geoapify_key  
}

# Print a message to follow up on the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"  # Example of filter parameter using a circle with the specified radius around the coordinates
    params["bias"] = f"proximity:{lng},{lat}"  # Example of bias parameter using the specific coordinates

    # Make an API request using the params dictionary
    response = requests.get("https://api.geoapify.com/v2/places", params=params)
    
    # Check if the API request was successful
    if response.status_code == 200:
        # Convert the API response to JSON format
        name_address = response.json()
        
        # Check if 'results' key is present in the JSON response
        try:
            # Extract the hotel name and address from the API response
           # hotel_name = nhotel_df.loc[index, "Hotel Name"]
           # hotel_address = hotel_df.loc[index, "Hotel Name"]
            # Store the hotel name in the hotel_df DataFrame
            hotel_df.loc[index, "Hotel_Name"] = name_address["features"][0]["properties"]["name"]
        except:
            # If no hotel is found, set the hotel name as "No hotel found"
            hotel_df.loc[index, "Hotel_Name"] = "No hotel found"
    else:
        # Handle API request failure
        print(f"API request failed for city: {hotel_df.loc[index, 'City']}")

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_Name']}")

# Display sample data
hotel_df


Starting hotel search
surbo - nearest hotel: Hotel Aloisi
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
stanley - nearest hotel: Hotel 52
viedma - nearest hotel: Níjar
tolanaro - nearest hotel: Hôtel Mahavokey
isafjordur - nearest hotel: Hótel Horn
reggane - nearest hotel: No hotel found
luanda - nearest hotel: Hotel Forum
prado - nearest hotel: Pousada Recanto do Prado
maceio - nearest hotel: Maceió Hostel e Pousada
hawaiian paradise park - nearest hotel: No hotel found
waitangi - nearest hotel: Hotel Chathams
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
edinburgh of the seven seas - nearest hotel: No hotel found
mandan - nearest hotel: North Country Inn
kone - nearest hotel: Pacifik Appartels
tubmanburg - nearest hotel: No hotel found
bontang - nearest hotel: Hotel Sederhana
westport - nearest hotel: Norwalk Inn and Conference Center
invercargill - nearest hotel: The Grand
port-aux-francais - nearest hotel: Keravel
mount gambier - nearest hotel: The Old Mount

,City,Country,Lat,Lng,Humidity,Hotel_Name
0,surbo,IT,40.3994,18.1331,68,Hotel Aloisi
1,haiku-pauwela,US,20.9219,-156.3051,74,Inn At Mama's Fish House
2,stanley,GB,54.8680,-1.6985,60,Hotel 52
3,viedma,AR,-40.8135,-62.9967,28,Níjar
4,tolanaro,MG,-25.0319,46.9987,94,Hôtel Mahavokey
...,...,...,...,...,...,...
545,diego de almagro,CL,-26.3667,-70.0500,14,Residencial Hambyll
546,el reno,US,35.5323,-97.9550,57,No hotel found
547,shingu,JP,33.7333,135.9833,84,ホテルニューパレス
548,lufkin,US,31.3382,-94.7291,46,Sun-N-Pines Motel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

import hvplot.pandas  # Import hvPlot for Pandas

# Configure the map plot_2
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    hover_cols=["Hotel_Name", "City", "Country"],
    size=10,
    color="red"
)

# Display the map plot_2
map_plot_2


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel_Name,City,Country)